In [6]:
pip install rdkit-pypi

Note: you may need to restart the kernel to use updated packages.


In [8]:
from rdkit import Chem
## Read the file
supplier = Chem.SDMolSupplier('/kaggle/input/sdfmutagenicity/final_sdfdata.sdf')
len(supplier)

11696

In [9]:
import numpy as np
from rdkit.Chem import AllChem
infos = [{} for mol in supplier]
# info = {} # will be mutated in the next function
## calculate the Morgan Fingerprints for every molecule in the supplier with radius 3 and nBits 2048
fingerprints = np.array([AllChem.GetMorganFingerprintAsBitVect(supplier[i], 3, nBits=2048, bitInfo=infos[i])
                         for i in range(len(supplier))])
infos = np.array(infos)
print(fingerprints.shape)
print(infos.shape)


(11696, 2048)
(11696,)


In [16]:
y=[]
for i in range(11696):
    for mol in supplier[i].GetProp("Y"):
        y.append(int(mol))

In [17]:
print(y)

[1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 

In [18]:
print(fingerprints[0])

[0 0 0 ... 0 0 0]


In [24]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from joblib import dump
supplier = Chem.SDMolSupplier('/kaggle/input/sdfmutagenicity/final_sdfdata.sdf')
fingerprints = np.array([AllChem.GetMorganFingerprintAsBitVect(supplier[i], 3, nBits=2048, bitInfo=infos[i])
                         for i in range(len(supplier))])
y=[]
for i in range(11696):
    for mol in supplier[i].GetProp("Y"):
        y.append(int(mol))
X_train, X_test, y_train, y_test = train_test_split(fingerprints, y, test_size=0.2, random_state=42)

# Train an SVM model
rf_model = RandomForestClassifier()
# Train the model
rf_model.fit(X_train, y_train)
dump(rf_model, 'mutagenicitysdf.joblib')
# Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9303418803418804
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.92      0.92       989
           1       0.94      0.94      0.94      1351

    accuracy                           0.93      2340
   macro avg       0.93      0.93      0.93      2340
weighted avg       0.93      0.93      0.93      2340



In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from joblib import dump
supplier = Chem.SDMolSupplier('/kaggle/input/sdfmutagenicity/final_sdfdata.sdf')
fingerprints = np.array([AllChem.GetMorganFingerprintAsBitVect(supplier[i], 3, nBits=2048, bitInfo=infos[i])
                         for i in range(len(supplier))])

X_train, X_test, y_train, y_test = train_test_split(fingerprints, y, test_size=0.2, random_state=42)

# Train an SVM model
rf_model = RandomForestClassifier()
# Train the model
rf_model.fit(X_train, y_train)
dump(rf_model, 'mutagenicitysdf.joblib')
# Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

In [22]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from joblib import dump
X_train, X_test, y_train, y_test = train_test_split(fingerprints, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier()
# Train the model
rf_model.fit(X_train, y_train)
dump(rf_model, 'mutagenicitycsv.joblib')
# Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9290598290598291
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.92      0.92       989
           1       0.94      0.94      0.94      1351

    accuracy                           0.93      2340
   macro avg       0.93      0.93      0.93      2340
weighted avg       0.93      0.93      0.93      2340

Accuracy: 0.885042735042735
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.88      0.87       989
           1       0.91      0.89      0.90      1351

    accuracy                           0.89      2340
   macro avg       0.88      0.88      0.88      2340
weighted avg       0.89      0.89      0.89      2340



In [4]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pprint
import pickle

#  load data

In [5]:
from rdkit import Chem
## Read the file
supplier = Chem.SDMolSupplier('/content/final_sdfdata.sdf')
len(supplier)


KeyboardInterrupt



# preprocessing

In [ ]:
import matplotlib.pyplot as plt

# Calculate counts of each class
class_counts = df['Y'].value_counts().sort_index()

# Replace class labels with 'negative' and 'positive'
class_labels = ['Non-mutagenic', 'Mutagentic']



# first model

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load the DataFrame
df = pd.read_csv('/kaggle/input/mutagenicity/mutaFull.csv')

# Generate Morgan fingerprints for each SMILES string
def generate_morgan_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)  # Radius 2
    return morgan_fp

# Generate fingerprints for all SMILES strings
X = [generate_morgan_fingerprint(smiles) for smiles in df['Drug']]

# Use class labels as target variable
y = df['Y'].tolist()

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an SVM model
svm_model = SVC()

# Train the model
svm_model.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


# model2

In [ ]:
# ////////////////////////////////////////////////////////
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from joblib import dump
# Load the DataFrame
df = pd.read_csv('/kaggle/input/mutagenicity/mutaFull.csv')

# Generate Morgan fingerprints for each SMILES string
def generate_morgan_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2,nBits=1024, )  # Radius 2
    return morgan_fp

# Generate fingerprints for all SMILES strings
X = [generate_morgan_fingerprint(smiles) for smiles in df['Drug']]

# Use class labels as the target variable
y = df['Y'].tolist()

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest classifier
rf_model = RandomForestClassifier()

# Train the model
rf_model.fit(X_train, y_train)
dump(rf_model, 'mutagenicitycsv.joblib')
# Evaluate the model
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


# model3 best GetMorganFingerprintAsBitVect with randomforest 

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the DataFrame
df = pd.read_csv('/kaggle/input/mutagenicity/mutaFull.csv')

# Generate Morgan fingerprints for each SMILES string
def generate_morgan_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024, useChirality=True, useBondTypes=True)
    return morgan_fp

# Generate fingerprints for all SMILES strings
X = [generate_morgan_fingerprint(smiles) for smiles in df['Drug']]

# Use class labels as the target variable
y = df['Y'].tolist()

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest classifier
rf_model = RandomForestClassifier()

# Train the model
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


# model4 RDKit fingerprint with random forest 

In [ ]:
# ////////////////////////////////////////////////////////////////////////////////////
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the DataFrame
df = pd.read_csv('/kaggle/input/mutagenicity/mutaFull.csv')

# Generate RDKit fingerprints for each SMILES string
def generate_rdkit_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    rdkit_fp = AllChem.RDKFingerprint(mol,fpSize=1024)
    return rdkit_fp

# Generate fingerprints for all SMILES strings
X = [generate_rdkit_fingerprint(smiles) for smiles in df['Drug']]

# Use class labels as the target variable
y = df['Y'].tolist()

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest classifier
rf_model = RandomForestClassifier()

# Train the model
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


# model5 GenMACCSKeys RandomForestClassifier 

In [ ]:
# //////////////////////////////////////////////////////////////////////////
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the DataFrame
df = pd.read_csv('/kaggle/input/mutagenicity/mutaFull.csv')

# Generate MACCS fingerprints for each SMILES string
def generate_maccs_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    maccs_fp = MACCSkeys.GenMACCSKeys(mol)
    return maccs_fp

# Generate fingerprints for all SMILES strings
X = [generate_maccs_fingerprint(smiles) for smiles in df['Drug']]

# Use class labels as the target variable
y = df['Y'].tolist()

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest classifier
rf_model = RandomForestClassifier()

# Train the model
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


# generate_avalon_fingerprint

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Avalon.pyAvalonTools import GetAvalonFP
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the DataFrame
df = pd.read_csv('/kaggle/input/mutagenicity/mutaFull.csv')

# Generate Avalon fingerprints for each SMILES string
def generate_avalon_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    avalon_fp = GetAvalonFP(mol)
    return avalon_fp

# Generate fingerprints for all SMILES strings
X = [generate_avalon_fingerprint(smiles) for smiles in df['Drug']]

# Use class labels as the target variable
y = df['Y'].tolist()

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest classifier
rf_model = RandomForestClassifier()

# Train the model
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)
